# Clean Questionnaire Data Pre/Post

In [129]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [131]:
deploy_type = "local"

In [132]:
config_dict = json.load(Path("../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

PosixPath('/Volumes/luca_ssd/Study_Data/2022_05_AP01_Macro')

In [133]:
code_mapping = pd.read_csv(base_path.joinpath("data_tabular/extras/code_to_number_mapping.csv"))
code_mapping = code_mapping.set_index("Code")
code_mapping.head()

,subject
Code,
BG05W,VP_01
HZ03B,VP_02
NW15N,VP_03
KS08F,VP_04
FA01B,VP_05


## Load Pre Questionnaire Data and Map Code to VP-ID

In [134]:
quest_data = pd.read_excel(base_path.joinpath("data_tabular/questionnaires/cleaned/unipark_pre.xlsx"))
quest_data = quest_data.rename(columns={"VPN_Kennung": "Code", "Tag": "day"})
quest_data = quest_data.set_index("Code")

quest_data = quest_data.join(code_mapping)

# add T before day column
quest_data["day"] = "T" + quest_data["day"].astype(str)

quest_data = quest_data.dropna(subset=["subject"]).set_index(["subject", "day"])

quest_data = quest_data.filter(like="Pre")

# put pre to the end

quest_data.columns = quest_data.columns.str.replace("Pre_", "")

quest_data.columns = quest_data.columns + "_pre"

quest_data

STADI_1_pre  STADI_2_pre  STADI_3_pre  STADI_4_pre  STADI_5_pre  \
subject day                                                                    
VP_09   T1             1            3            4            1            3   
        T2             1            1            3            1            2   
VP_25   T1             2            3            2            1            1   
        T2             1            3            2            1            1   
VP_27   T1             2            3            3            2            3   
...                  ...          ...          ...          ...          ...   
VP_20   T2             2            2            3            1            3   
VP_29   T1             1            2            3            1            1   
        T2             1            2            2            1            2   
VP_28   T1             4            2            2            1            1   
        T2             2            2            2            1            1   

             STADI_6_pre  STADI_7_pre  STADI_8_pre  STADI_9_pre  STADI_10_pre  \
subject day                                                                     
VP_09   T1             3            4            1            3             4   
        T2             2            3            1            1             2   
VP_25   T1             3            2            1            1             2   
        T2             3            2            2            1             2   
VP_27   T1             4            3            2            4             4   
...                  ...          ...          ...          ...           ...   
VP_20   T2             2            2            2            3             2   
VP_29   T1             2            2            1            1             1   
        T2             2            2            1            2             2   
VP_28   T1             3            2            1            1             3   
        T2             2            2            2            2             2   

             ...  SSSQ_15_pre  SSSQ_16_pre  SSSQ_17_pre  SSSQ_18_pre  \
subject day  ...                                                       
VP_09   T1   ...            5            4            3            2   
        T2   ...            3            3            3            2   
VP_25   T1   ...            5            1            2            3   
        T2   ...            4            1            2            2   
VP_27   T1   ...            5            5            3            5   
...          ...          ...          ...          ...          ...   
VP_20   T2   ...            2            1            3            2   
VP_29   T1   ...            3            2            3            2   
        T2   ...            3            2            2            2   
VP_28   T1   ...            4            1            2            4   
        T2   ...            2            1            1            2   

             SSSQ_19_pre  SSSQ_20_pre  SSSQ_21_pre  SSSQ_22_pre  SSSQ_23_pre  \
subject day                                                                    
VP_09   T1             3            3            2            4            3   
        T2             2            2            3            4            4   
VP_25   T1             2            1            2            2            2   
        T2             2            1            3            2            1   
VP_27   T1             2            4            3            3            4   
...                  ...          ...          ...          ...          ...   
VP_20   T2             2            2            3            3            2   
VP_29   T1             3            3            3            4            1   
        T2             2            2            3            3            2   
VP_28   T1             4            4            1            3            5   
        T2           

## Load Post Questionnaire Data and Join With Pre

In [135]:
quest_data_post = pd.read_excel(base_path.joinpath("data_tabular/questionnaires/cleaned/unipark_post.xlsx"))
quest_data_post = quest_data_post.rename(columns={"VPN_Kennung": "Code", "Tag": "day"})
quest_data_post = quest_data_post.set_index("Code")

quest_data_post = quest_data_post.join(code_mapping)

# add T before day column
quest_data_post["day"] = "T" + quest_data_post["day"].astype(str)

quest_data_post = quest_data_post.dropna(subset=["subject"]).set_index(["subject", "day"])

quest_data_post = quest_data_post.filter(like="Post")

# put post to the end

quest_data_post.columns = quest_data_post.columns.str.replace("Post_", "")

quest_data_post.columns = quest_data_post.columns + "_post"

quest_data_post

STADI_1_post  STADI_2_post  STADI_3_post  STADI_4_post  \
subject day                                                           
VP_09   T1              2             2             2             2   
        T2              1             2             3             1   
VP_25   T1              1             2             2             1   
        T2              1             3             2             1   
VP_27   T1              4             1             4             1   
...                   ...           ...           ...           ...   
VP_20   T2              1             1             3             1   
VP_29   T1              2             3             2             1   
        T2              1             1             2             1   
VP_28   T1              3             1             3             1   
        T2              3             1             2             1   

             STADI_5_post  STADI_6_post  STADI_7_post  STADI_8_post  \
subject day                                                           
VP_09   T1              2             2             2             2   
        T2              2             2             4             1   
VP_25   T1              1             3             2             2   
        T2              1             3             2             2   
VP_27   T1              4             1             3             1   
...                   ...           ...           ...           ...   
VP_20   T2              1             1             3             1   
VP_29   T1              2             2             2             1   
        T2              1             2             2             1   
VP_28   T1              1             2             3             1   
        T2              1             2             2             2   

             STADI_9_post  STADI_10_post  ...  SSGS_6_post  SSGS_7_post  \
subject day                               ...                             
VP_09   T1              2              2  ...          2.0          2.0   
        T2              2              1  ...          NaN          NaN   
VP_25   T1              1              3  ...          1.0          2.0   
        T2              1              2  ...          1.0          2.0   
VP_27   T1              4              4  ...          5.0          3.0   
...                   ...            ...  ...          ...          ...   
VP_20   T2              2              1  ...          2.0          1.0   
VP_29   T1              2              1  ...          3.0          2.0   
        T2              1              1  ...          1.0          4.0   
VP_28   T1              2              2  ...          4.0          1.0   
        T2              2              2  ...          3.0          2.0   

             SSGS_8_post  SSGS_9_post  SSGS_10_post  SSGS_11_post  \
subject day                                                         
VP_09   T1           2.0          2.0           2.0           2.0   
        T2           NaN          NaN           NaN           NaN   
VP_25   T1           1.0          1.0           1.0           1.0   
        T2           1.0          1.0           1.0           1.0   
VP_27   T1           2.0          5.0           2.0           3.0   
...                  ...          ...           ...           ...   
VP_20   T2           1.0          1.0           4.0           1.0   
VP_29   T1           1.0          2.0           2.0           1.0   
        T2           1.0          1.0           4.0           1.0   
VP_28   T1           1.0          2.0           1.0           1.0   
        T2           1.0          1.0           1.0           1.0   

             SSGS_12_post  SSGS_13_post  SSGS_14_post  SSGS_15_post  
subject day                                                          
VP_09   T1            2.0           2.0           2.0           2.0  
        T2            NaN           NaN           NaN           NaN  
VP_25   T1 

In [136]:
quest_data_all = quest_data.join(quest_data_post)

In [137]:
quest_data_all.replace(-77, np.nan, inplace=True)

In [138]:
quest_data_all

STADI_1_pre  STADI_2_pre  STADI_3_pre  STADI_4_pre  STADI_5_pre  \
subject day                                                                    
VP_09   T1             1            3            4            1            3   
        T2             1            1            3            1            2   
VP_25   T1             2            3            2            1            1   
        T2             1            3            2            1            1   
VP_27   T1             2            3            3            2            3   
...                  ...          ...          ...          ...          ...   
VP_20   T2             2            2            3            1            3   
VP_29   T1             1            2            3            1            1   
        T2             1            2            2            1            2   
VP_28   T1             4            2            2            1            1   
        T2             2            2            2            1            1   

             STADI_6_pre  STADI_7_pre  STADI_8_pre  STADI_9_pre  STADI_10_pre  \
subject day                                                                     
VP_09   T1             3            4            1            3             4   
        T2             2            3            1            1             2   
VP_25   T1             3            2            1            1             2   
        T2             3            2            2            1             2   
VP_27   T1             4            3            2            4             4   
...                  ...          ...          ...          ...           ...   
VP_20   T2             2            2            2            3             2   
VP_29   T1             2            2            1            1             1   
        T2             2            2            1            2             2   
VP_28   T1             3            2            1            1             3   
        T2             2            2            2            2             2   

             ...  SSGS_6_post  SSGS_7_post  SSGS_8_post  SSGS_9_post  \
subject day  ...                                                       
VP_09   T1   ...          2.0          2.0          2.0          2.0   
        T2   ...          NaN          NaN          NaN          NaN   
VP_25   T1   ...          1.0          2.0          1.0          1.0   
        T2   ...          1.0          2.0          1.0          1.0   
VP_27   T1   ...          5.0          3.0          2.0          5.0   
...          ...          ...          ...          ...          ...   
VP_20   T2   ...          2.0          1.0          1.0          1.0   
VP_29   T1   ...          3.0          2.0          1.0          2.0   
        T2   ...          1.0          4.0          1.0          1.0   
VP_28   T1   ...          4.0          1.0          1.0          2.0   
        T2   ...          3.0          2.0          1.0          1.0   

             SSGS_10_post  SSGS_11_post  SSGS_12_post  SSGS_13_post  \
subject day                                                           
VP_09   T1            2.0           2.0           2.0           2.0   
        T2            NaN           NaN           NaN           NaN   
VP_25   T1            1.0           1.0           1.0           1.0   
        T2            1.0           1.0           1.0           3.0   
VP_27   T1            2.0           3.0           4.0           2.0   
...                   ...           ...           ...           ...   
VP_20   T2            4.0           1.0           1.0           4.0   
VP_29   T1            2.0           1.0           3.0           3.0   
        T2            4.0           1.0           1.0           4.0   
VP_28   T1            1.0           1.0           1.0           1.0   
        T2            1.0           1.0           1.0           2.0   

             SSGS_14_post  SSGS_15_post  
subject day    

In [139]:
# map day to condition

day_to_condition = pd.read_csv(base_path.joinpath("data_tabular/extras/condition_order.csv"))

# convert to long format
day_to_condition_long = day_to_condition.melt(id_vars="subject", value_vars=["T1", "T2"], value_name="condition", var_name="day")


In [140]:
day_to_condition_long.set_index(["subject", "day"], inplace=True)


In [141]:
quest_data_all = quest_data_all.join(day_to_condition_long).reset_index()
quest_data_all.set_index(["subject", "condition"], inplace=True)
quest_data_all.drop(columns="day", inplace=True)

In [142]:
# add trailing zeros

column_names_split = quest_data_all.columns.str.split("_")

quest_data_all.columns = column_names_split.str[0] + "_" + column_names_split.str[1].str.zfill(2) + "_" + column_names_split.str[2]

In [143]:
quest_data_all

STADI_01_pre  STADI_02_pre  STADI_03_pre  STADI_04_pre  \
subject condition                                                           
VP_09   tsst                  1             3             4             1   
        ftsst                 1             1             3             1   
VP_25   ftsst                 2             3             2             1   
        tsst                  1             3             2             1   
VP_27   tsst                  2             3             3             2   
...                         ...           ...           ...           ...   
VP_20   ftsst                 2             2             3             1   
VP_29   tsst                  1             2             3             1   
        ftsst                 1             2             2             1   
VP_28   tsst                  4             2             2             1   
        ftsst                 2             2             2             1   

                   STADI_05_pre  STADI_06_pre  STADI_07_pre  STADI_08_pre  \
subject condition                                                           
VP_09   tsst                  3             3             4             1   
        ftsst                 2             2             3             1   
VP_25   ftsst                 1             3             2             1   
        tsst                  1             3             2             2   
VP_27   tsst                  3             4             3             2   
...                         ...           ...           ...           ...   
VP_20   ftsst                 3             2             2             2   
VP_29   tsst                  1             2             2             1   
        ftsst                 2             2             2             1   
VP_28   tsst                  1             3             2             1   
        ftsst                 1             2             2             2   

                   STADI_09_pre  STADI_10_pre  ...  SSGS_06_post  \
subject condition                              ...                 
VP_09   tsst                  3             4  ...           2.0   
        ftsst                 1             2  ...           NaN   
VP_25   ftsst                 1             2  ...           1.0   
        tsst                  1             2  ...           1.0   
VP_27   tsst                  4             4  ...           5.0   
...                         ...           ...  ...           ...   
VP_20   ftsst                 3             2  ...           2.0   
VP_29   tsst                  1             1  ...           3.0   
        ftsst                 2             2  ...           1.0   
VP_28   tsst                  1             3  ...           4.0   
        ftsst                 2             2  ...           3.0   

                   SSGS_07_post  SSGS_08_post  SSGS_09_post  SSGS_10_post  \
subject condition                                                           
VP_09   tsst                2.0           2.0           2.0           2.0   
        ftsst               NaN           NaN           NaN           NaN   
VP_25   ftsst               2.0           1.0           1.0           1.0   
        tsst                2.0           1.0           1.0           1.0   
VP_27   tsst                3.0           2.0           5.0           2.0   
...                         ...           ...           ...           ...   
VP_20   ftsst               1.0           1.0           1.0           4.0   
VP_29   tsst                2.0           1.0           2.0           2.0   
        ftsst               4.0           1.0           1.0           4.0   
VP_28   tsst                1.0           1.0           2.0           1.0   
        ftsst               2.0           1.0           1.0           1.0   

                   SSGS_11_post  SSGS_12_post  SSGS_13_post  SSGS_14_post  \
subject condition                                  

In [144]:
# unstack condition
quest_data_all = quest_data_all.unstack("condition")

In [146]:
quest_data_all.sort_index(axis=1, level=1, inplace=True)

In [148]:
# squeeze multiindex
quest_data_all.columns = ["_".join(a) for a in quest_data_all.columns.to_flat_index()]

In [149]:
quest_data_all

,PANAS_01_post_ftsst,PANAS_01_pre_ftsst,PANAS_02_post_ftsst,PANAS_02_pre_ftsst,PANAS_03_post_ftsst,PANAS_03_pre_ftsst,PANAS_04_post_ftsst,PANAS_04_pre_ftsst,PANAS_05_post_ftsst,PANAS_05_pre_ftsst,...,STADI_16_post_tsst,STADI_16_pre_tsst,STADI_17_post_tsst,STADI_17_pre_tsst,STADI_18_post_tsst,STADI_18_pre_tsst,STADI_19_post_tsst,STADI_19_pre_tsst,STADI_20_post_tsst,STADI_20_pre_tsst
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,3,3,1,2,3,3,3,1,1,1,...,3,1,3,2,3,2,3,3,1,1
VP_02,3,3,1,1,3,2,2,1,1,1,...,1,1,4,3,3,2,2,4,2,1
VP_03,3,2,1,1,2,4,3,2,1,1,...,1,1,2,2,2,1,2,3,1,1
VP_04,3,3,2,2,4,4,3,2,1,1,...,3,1,1,2,2,2,2,3,3,1
VP_05,3,3,2,2,3,3,2,3,1,1,...,2,1,1,2,3,2,2,1,1,1
VP_06,4,3,1,2,4,4,3,3,1,1,...,1,1,1,1,2,1,2,4,1,1
VP_07,3,4,1,1,4,4,4,3,1,1,...,1,1,2,2,1,1,4,3,1,1
VP_08,5,5,2,2,4,4,4,4,1,2,...,1,1,2,2,2,2,2,4,1,1
VP_09,4,3,2,2,4,4,3,2,1,1,...,2,1,2,3,2,3,2,3,2,1


## TSGS

In [ ]:
quest_data.filter(like="TSGS").to_csv(base_path.joinpath("data_tabular/questionnaires/final/tsgs.csv"))